In [39]:
import pandas as pd
df = pd.read_excel('C:\\Users\\goodluck\\Desktop\\0712.xlsx')

In [40]:
import pandas as pd
import numpy as np
import pickle
import time 
import os
from datetime import datetime
from operator import itemgetter
import time

# 需要什么读取什么不要一次性全都读取了，影响程序速度

data_folder_path = '../'
raw_data_path = "../raw_data"
storage_path = "../database_storage"
daily_path = "../daily"
DTBS_path = os.path.join(storage_path, "DTBS.pkl")
DCBS_path = os.path.join(storage_path, "DCBS.pkl")

time1 = time.time()

with open('database_ByZCG\\DCBS_DataFrame.pkl', 'rb') as f:  
    DCBS_DataFrame = pickle.load(f)
    

with open('database_ByZCG\\DTBS_DataFrame.pkl', 'rb') as f:  
    DTBS_DataFrame = pickle.load(f)

    
# 只提取A区中存活的转债的时间序列信息
A_DF = DTBS_DataFrame['A']
A_DF = A_DF[A_DF['ia']==1].copy()

# 提取B区信息
B_DF = DTBS_DataFrame['B']

In [41]:
print(df)

# 时间改成今天或者你想要获取的某一天
A_DF = A_DF[A_DF['date']=='2023-08-28'].copy()
A_DF = pd.merge(A_DF,B_DF[['bond_code','cat1','cat2']],on='bond_code',how='left')
print(A_DF)

temp_df = pd.merge(df,A_DF[['bond_code','dp','cpr', 'dl', 'pd', 'ytm', 'bl', 'trt','cat1', 'cat2', 'yl']],left_on='Unnamed: 1',right_on='bond_code',how='left')
print(temp_df)

    Unnamed: 0 Unnamed: 1 Unnamed: 2  \
0          NaN  128105.SZ       长集转债   
1          NaN  123014.SZ       亚太转债   
2          NaN  128017.SZ       金禾转债   
3          NaN  113524.SH       奇精转债   
4          NaN  123169.SZ       正海转债   
..         ...        ...        ...   
178        NaN  127028.SZ       英特转债   
179        NaN  127076.SZ       中宠转2   
180        NaN  128136.SZ       立讯转债   
181        NaN  113659.SH       莱克转债   
182        NaN  123197.SZ       光力转债   

                                                    热点     其它  其它2  收盘价  \
0                                                   电力    NaN  NaN  NaN   
1                                                汽车零部件    NaN  NaN  NaN   
2                                                   代糖    NaN  NaN  NaN   
3                                            汽车零部件（液压）    NaN  NaN  NaN   
4                                                 稀土永磁    NaN  NaN  NaN   
..                                                 ...    ...  ...  .

In [42]:
# 使用drop方法删除多列
temp_df = temp_df.drop(columns=['bond_code','收盘价','转股溢价率','双低','平底溢价率','YTM','剩余规模','申万一级','申万二级','剩余年限'])
temp_df.rename(columns={'Unnamed: 1' : '转债代码',
                        'Unnamed: 2' : '转债名称',
                        'cpr' : '转股溢价率',
                        'dl' : '双低',
                        'dp' : '收盘价',
                        'change':'涨跌幅',
                        'sn' : '正股名称',
                        'bl' : '债券余额（亿）',
                        'yl' : '剩余期限（年）',
                        'trt' : '换手率',
                        'pd' : '平底溢价率',
                        'cat1' : '申万一级',
                        'cat2' : '申万二级',
                        'ytm' : '到期收益率'},inplace=True)

temp_df['计数（查看这只转债是第几次出现的）'] = temp_df.groupby('转债代码').cumcount() + 1

In [43]:
temp_df
temp_df.to_excel('0828结果.xlsx',index=False)